import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=480
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Haptics(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (155, 1093)
first_test_shape= (308, 1093)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [18 34 34 36 33]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -11.147
first_train_sample=
 [ 5.      -1.0475   0.54834 ...  0.89676 -2.6913  -8.1963 ]
1 18	2 34	3 34	4 36	5 33	

In [3]:
np.shape(ecg)

(155, 481)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 18	2 34	3 34	4 36	5 33	
 max = 36


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (18, 481)
Up to class  2 train shape =  (52, 481)
Up to class  3 train shape =  (86, 481)
Up to class  4 train shape =  (122, 481)
Up to class  5 train shape =  (155, 481)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 18 34 34 36 33]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.,
       5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.597467569572298
min magnitude class 1  =  -13.17299850668317
after normalizing >>
max magnitude class 1  =  1.2596009116229263
min magnitude class 1  =  -1.8343638646323206

 cls 2  >> 
max magnitude class 2  =  2.7396485902086973
min magnitude class 2  =  -7.29747643241007
after normalizing >>
max magnitude class 2  =  0.9361413884937131
min magnitude class 2  =  -0.811393602518538

 cls 3  >> 
max magnitude class 3  =  2.921229641073384
min magnitude class 3  =  -8.077613775922405
after normalizing >>
max magnitude class 3  =  0.9677559433962111
min magnitude class 3  =  -0.9472210732844096

 cls 4  >> 
max magnitude class 4  =  2.566649466280018
min magnitude class 4  =  -6.7000715338116725
after normalizing >>
max magnitude class 4  =  0.9060210082256042
min magnitude class 4  =  -0.707381152563975

 cls 5  >> 
max magnitude class 5  =  3.059927687644744
min magnitude class 5  =  -7.133883777493334
after normalizing >>
max magnitude class 5 

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.8343638646323206
max tst = 1.2596009116229263


In [14]:
len(wndws_test)

308

In [15]:
np.shape(wndws_test)

(308, 481)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(18, 481)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (155, 480)
y_train =>  (155,)
X_test  =>  (308, 480)
y_test  =>  (308,)
X_valid  =>  (155, 480)
y_valid  =>  (155,)


In [22]:
y_train=np.array(y_train+1)
y_test=np.array(y_test+1)
y_valid=np.array(y_valid+1)

In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [24]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
8/8 [==============================] - 3s 53ms/step - loss: 1.7776 - accuracy: 0.1935 - val_loss: 1.6080 - val_accuracy: 0.2194
Epoch 2/1000
8/8 [==============================] - 0s 13ms/step - loss: 1.6573 - accuracy: 0.2129 - val_loss: 1.6174 - val_accuracy: 0.2194
Epoch 3/1000
8/8 [==============================] - 0s 13ms/step - loss: 1.6235 - accuracy: 0.2839 - val_loss: 1.6191 - val_accuracy: 0.2129
Epoch 4/1000
8/8 [==============================] - 0s 14ms/step - loss: 1.6123 - accuracy: 0.1935 - val_loss: 1.6023 - val_accuracy: 0.3548
Epoch 5/1000
8/8 [==============================] - 0s 13ms/step - loss: 1.6271 - accuracy: 0.2452 - val_loss: 1.5825 - val_accuracy: 0.3097
Epoch 6/1000
8/8 [==============================] - 0s 13ms/step - loss: 1.6261 - accuracy: 0.1871 - val_loss: 1.5993 - val_accuracy: 0.3548
Epoch 7/1000
8/8 [==============================] - 0s 13ms/step - loss: 1.5945 - accuracy: 0.2968 - val_loss: 1.5783 - val_accuracy: 0.2774
Epoch 8/1000


Epoch 59/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.6150 - accuracy: 0.7742 - val_loss: 0.4578 - val_accuracy: 0.8194
Epoch 60/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.6397 - accuracy: 0.7548 - val_loss: 0.4749 - val_accuracy: 0.8387
Epoch 61/1000
8/8 [==============================] - 0s 14ms/step - loss: 0.5869 - accuracy: 0.7484 - val_loss: 0.4213 - val_accuracy: 0.8323
Epoch 62/1000
8/8 [==============================] - 0s 14ms/step - loss: 0.5969 - accuracy: 0.7548 - val_loss: 0.5802 - val_accuracy: 0.7677
Epoch 63/1000
8/8 [==============================] - 0s 13ms/step - loss: 0.6204 - accuracy: 0.7484 - val_loss: 0.4778 - val_accuracy: 0.8000
Epoch 64/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.5708 - accuracy: 0.7806 - val_loss: 0.3195 - val_accuracy: 0.9097
Epoch 65/1000
8/8 [==============================] - 0s 14ms/step - loss: 0.3934 - accuracy: 0.8387 - val_loss: 0.5970 - val_accuracy: 0.7613
Epoch 

                                                                 
 max_pooling1d_1 (MaxPooling  (None, 52, 128)          0         
 1D)                                                             
                                                                 
 conv1d_3 (Conv1D)           (None, 50, 128)           49280     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 16, 128)          0         
 1D)                                                             
                                                                 
 conv1d_4 (Conv1D)           (None, 14, 256)           98560     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 4, 256)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 1024)              0         
          

In [25]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [26]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 2ms/step
array([[13,  9, 24,  6,  8],
       [ 0, 33,  9, 12,  4],
       [ 3, 11, 34,  6,  5],
       [ 5, 11,  7, 31, 10],
       [15,  6, 12,  8, 26]], dtype=int64)
              precision    recall  f1-score   support

           2       0.36      0.22      0.27        60
           3       0.47      0.57      0.52        58
           4       0.40      0.58      0.47        59
           5       0.49      0.48      0.49        64
           6       0.49      0.39      0.43        67

    accuracy                           0.44       308
   macro avg       0.44      0.45      0.44       308
weighted avg       0.44      0.44      0.44       308



In [27]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
8/8 [==============================] - 2s 88ms/step - loss: 1.9087 - accuracy: 0.1871 - val_loss: 1.6446 - val_accuracy: 0.2323
Epoch 2/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6358 - accuracy: 0.2710 - val_loss: 1.6087 - val_accuracy: 0.2194
Epoch 3/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6371 - accuracy: 0.1871 - val_loss: 1.6300 - val_accuracy: 0.2194
Epoch 4/1000
8/8 [==============================] - 0s 16ms/step - loss: 1.6088 - accuracy: 0.2903 - val_loss: 1.6115 - val_accuracy: 0.2194
Epoch 5/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6134 - accuracy: 0.1871 - val_loss: 1.6040 - val_accuracy: 0.2194
Epoch 6/1000
8/8 [==============================] - 0s 14ms/step - loss: 1.6321 - accuracy: 0.2065 - val_loss: 1.5933 - val_accuracy: 0.2194
Epoch 7/1000
8/8 [==============================] - 0s 15ms/step - loss: 1.6350 - accuracy: 0.1742 - val_loss: 1.5947 - val_accuracy: 0.2323
Epoch 8/1000


Epoch 59/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.6835 - accuracy: 0.7161 - val_loss: 0.5952 - val_accuracy: 0.8000
Epoch 60/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.6674 - accuracy: 0.7355 - val_loss: 0.7565 - val_accuracy: 0.6581
Epoch 61/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.7274 - accuracy: 0.7032 - val_loss: 0.8958 - val_accuracy: 0.6258
Epoch 62/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.8975 - accuracy: 0.6516 - val_loss: 0.6892 - val_accuracy: 0.7613
Epoch 63/1000
8/8 [==============================] - 0s 17ms/step - loss: 0.6117 - accuracy: 0.7419 - val_loss: 0.5365 - val_accuracy: 0.8129
Epoch 64/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.6442 - accuracy: 0.7548 - val_loss: 0.7585 - val_accuracy: 0.6774
Epoch 65/1000
8/8 [==============================] - 0s 17ms/step - loss: 0.6545 - accuracy: 0.7419 - val_loss: 0.4673 - val_accuracy: 0.8323
Epoch 

8/8 [==============================] - 0s 16ms/step - loss: 0.0299 - accuracy: 1.0000 - val_loss: 0.0190 - val_accuracy: 1.0000
Epoch 117/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.0144 - accuracy: 1.0000 - val_loss: 0.0102 - val_accuracy: 1.0000
Epoch 118/1000
8/8 [==============================] - 0s 16ms/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.0077 - val_accuracy: 1.0000
Epoch 119/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.0068 - val_accuracy: 1.0000
Epoch 120/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.2785 - val_accuracy: 0.8968
Epoch 121/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.8493 - accuracy: 0.7226 - val_loss: 0.0721 - val_accuracy: 0.9742
Epoch 122/1000
8/8 [==============================] - 0s 15ms/step - loss: 0.0462 - accuracy: 0.9871 - val_loss: 0.0163 - val_accuracy: 1.0000
Epoch 123/1000

In [28]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

10/10 [==============================] - 0s 3ms/step
array([[ 6,  7, 16, 12, 19],
       [ 2, 30,  8, 16,  2],
       [ 5, 10, 35,  4,  5],
       [ 3, 12,  7, 32, 10],
       [ 8,  7,  7, 10, 35]], dtype=int64)
              precision    recall  f1-score   support

           2       0.25      0.10      0.14        60
           3       0.45      0.52      0.48        58
           4       0.48      0.59      0.53        59
           5       0.43      0.50      0.46        64
           6       0.49      0.52      0.51        67

    accuracy                           0.45       308
   macro avg       0.42      0.45      0.43       308
weighted avg       0.42      0.45      0.43       308

